<a href="https://colab.research.google.com/github/SamikshaSolanke/Netflix-Movie-Recommendation-System/blob/main/notebooks/ntbk4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [27]:
!pip install tmdbv3api

In [28]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'c353fd73554c1d566ea8a57b71508242'

In [6]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [9]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [10]:
tables = soup.find_all('table',class_='wikitable sortable')

In [11]:
len(tables)

4

In [12]:
type(tables[0])

bs4.element.Tag

In [13]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

<ipython-input-13-2803005693>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(str(tables[0]))[0]
<ipython-input-13-2803005693>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_html(str(tables[1]))[0]
<ipython-input-13-2803005693>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3 = pd.read_html(str(tables[2]))[0]
<ipython-input-13-2803005693>:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoide

In [14]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [15]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
...,...,...,...,...,...,...
273,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,[246]
274,D E C E M B E R,25,News of the World,Universal Pictures / Playtone / Perfect World ...,Paul Greengrass (director/screenplay); Luke Da...,[247]
275,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,[248]
276,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,[249]


In [17]:
df = df.rename(columns = {'Title' : 'movie_title'})

In [18]:
df['movie_title'] = df['movie_title'].str.lower()

In [20]:
def get_director(x):
  if " (director)" in x:
    return x.split(" (director)")[0]
  elif " (directors)" in x:
    return x.split(" (directors)")[0]
  else:
    return x.split(" (director/screenplay)")[0]

In [21]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [22]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.nan
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [31]:
df_2020 = df[['movie_title','Cast and crew']]

In [40]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    try:
        result = tmdb_movie.search(x)
        if not result:
          return np.nan
        else:
          # Access the id of the first result. Use a try-except for robustness.
          movie_id = result[0].id
          # Ensure tmdb object with api_key is initialized before running this cell
          response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
          response.raise_for_status() # Raise an exception for bad status codes
          data_json = response.json()
          if data_json.get('genres'): # Use .get() for safe access
              genre_str = " "
              for i in range(0,len(data_json['genres'])):
                  genres.append(data_json['genres'][i]['name'])
              return genre_str.join(genres)
          else:
              return np.nan
    except Exception as e:
        print(f"Error fetching genre for '{x}': {e}") # Print error for debugging
        return np.nan

In [41]:
df_2020['genres'] = df_2020['movie_title'].map(lambda x: get_genre(str(x)))

Error fetching genre for 'no films had been released on this date as resulting the closure of theaters caused by the covid-19 pandemic': attribute name must be string, not 'int'
Error fetching genre for 'the spongebob movie: sponge on the run (canadian theatrical release)': attribute name must be string, not 'int'


<ipython-input-41-1733152889>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['movie_title'].map(lambda x: get_genre(str(x)))


In [47]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [42]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-42-3646981363>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [43]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-43-463054939>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [44]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [45]:
df_2020

,movie_title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name
0,the grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Andrea Riseborough,Demián Bichir,John Cho
1,underwater,"William Eubank (director); Brian Duffield, Ada...",Drama,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,like a boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Tiffany Haddish,Rose Byrne,Salma Hayek
3,three christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Richard Gere,Peter Dinklage,Walton Goggins
4,inherit the viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...
273,we can be heroes,Robert Rodriguez (director/screenplay); Priyan...,Family Action Fantasy Comedy,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
274,news of the world,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure,Tom Hanks,Helena Zengel,NaN
275,one night in miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
276,promising young woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Carey Mulligan,Bo Burnham,Alison Brie


In [48]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [49]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [50]:
new_df20.isna().sum()

,0
director_name,0
actor_1_name,0
actor_2_name,5
actor_3_name,28
genres,2
movie_title,0
comb,29


In [51]:
new_df20 = new_df20.dropna(how='any')

In [52]:
old_df = pd.read_csv('/content/drive/MyDrive/dataset/final_data.csv')

In [55]:
final_df = pd.concat([new_df20, old_df], ignore_index=True)

In [56]:
final_df.to_csv('/content/drive/MyDrive/dataset/finalDataset.csv')